# Anaylsis of Ishigami Function

In [ ]:
import dill
import os
import chaospy as cp
import numpy as np
import math
import sys
import pathlib
import pandas as pd
import pickle
import time
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()

import uqef

In [ ]:
sys.path.insert(1, '/work/ga45met/mnt/linux_cluster_2/UQEF-Dynamic')
from uqef_dynamic.utils import utility
from uqef_dynamic.utils import uqPostprocessing
from uqef_dynamic.models.ishigami import IshigamiModel
from uqef_dynamic.models.ishigami import IshigamiStatistics


- [m1] MC (samples-based) with pick-freeze approach
- [m2] Saltelli/MC (samples-based) with rank-based approach
- [m3] gPCE+SC regression approach...
- [m4] gPCE+PSP with a full grid and polynomials of total-order
- [m5] gPCE+PSP with sparse grid and polynomials of total-order
- [m6] gPCE+PSP with a full grid and sparse polynomials (hyperbolic truncation)
- [m7] gPCE+PSP with sparse grid and sparse polynomials (hyperbolic truncation)



## (Simple) Running the Ishigami function

In [ ]:
a = 7
b = 0.1
ishigamiModelObject = IshigamiModel.IshigamiModel(configurationObject=None, a=a, b=b)

In [ ]:
x1 = cp.Uniform(-math.pi, math.pi)
x2 = cp.Uniform(-math.pi, math.pi)
x3 = cp.Uniform(-math.pi, math.pi)
joint_isghigami = cp.J(x1, x2, x3)
joint_isghigami_standard = cp.J(cp.Uniform(-1,1), cp.Uniform(-1,1), cp.Uniform(-1,1))

In [ ]:
coordinates = joint_isghigami.sample(10)
coordinates.shape

In [ ]:
# sime show-case how to run IshigamiModel for multiple set of parameters
coordinates = joint_isghigami.sample(10)
# results = ishigamiModelObject.run(i_s=range(coordinates.shape[1]), parameters=coordinates.T)
results = ishigamiModelObject(i_s=range(coordinates.shape[1]), parameters=coordinates.T)
results

In [ ]:
results[0][0]['result_time_series']

In [ ]:
ishigamiModelObject.list_qoi_column
ishigamiModelObject.time_column_name
ishigamiModelObject.index_column_name

In [ ]:
# processing the collected result
df_simulation_result, df_index_parameter_values, _, _, _, _ =  uqPostprocessing.uqef_dynamic_model_run_results_array_to_dataframe(results, 
 extract_only_qoi_columns=False, qoi_columns=ishigamiModelObject.list_qoi_column, 
 time_column_name= ishigamiModelObject.time_column_name, index_column_name= ishigamiModelObject.index_column_name)
df_simulation_result

In [ ]:
# Just to double-check if the order of parametres indeed corresponds to the order of model output...
coordinates = df_index_parameter_values[df_index_parameter_values[ishigamiModelObject.index_column_name]==0][["x1", "x2", "x3"]].values[0]
IshigamiModel.ishigami_func(coordinates, a_model_param=7, b_model_param=0.1)


In [ ]:
df_index_parameter_values

## Plotting the Function

In [ ]:
# Plotting the Ishigami Function
#IshigamiModel.ishigami_func(coordinates, a_model_param=7, b_model_param=0.1)
x1_array = [0.0, math.pi/4, math.pi/2, math.pi]
x2_array = [0.0, math.pi/4, math.pi/2, math.pi]
x3_array = [0.0, math.pi/4, math.pi/2, math.pi]
l = len(x3_array)
# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(0.25))
x = y = np.arange(-math.pi, math.pi, 0.05)
X, Y = np.meshgrid(x, y)
for i in range(l):
    ax = fig.add_subplot(1, l, i+1, projection='3d')
    zs = np.array(
        IshigamiModel.ishigami_func_vec((x1_array[i], np.ravel(X), np.ravel(Y)), a_model_param=7, b_model_param=0.1)
    )
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
    ax.set_xlabel('X2')
    ax.set_ylabel('X3')
    ax.set_zlabel(f'Ishigami(x2,x3)')
    ax.set_title(f'x1={x1_array[i]}')

# Add a color bar which maps values to colors.
# fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
x = y = np.arange(-math.pi, math.pi, 0.05)
X, Y = np.meshgrid(x, y)
zs = np.array(
        IshigamiModel.ishigami_func_vec((np.ravel(X), np.ravel(Y), math.pi), a_model_param=7, b_model_param=0.1)
)
Z = zs.reshape(X.shape)
Z_0 = np.zeros(X.shape)
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.add_trace(go.Surface(z=Z_0, x=X, y=Y))
fig.update_layout(scene = dict(
                    xaxis_title='x1',
                    yaxis_title='x2',
                    zaxis_title='ishigami(x1,x2)'),
                    width=650,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

# Analysis of a Single Output Produced by UQEF-Dynamics

In [ ]:
# TODO - change these paths/variables accordingly
scratch_dir = pathlib.Path("/work/ga45met/ishigami_runs/simulations_sep_2024")

workingDir = scratch_dir / "sc_full_p4_l8_ct07"
workingDir = scratch_dir / "sc_full_p4_q8"
workingDir = scratch_dir / "mc_100000_random"
workingDir = scratch_dir / "sc_full_p7_q14"

qoi_string="Value"
timestamp=0.0

In [ ]:
if not workingDir.is_dir():
    print("Watch out - The path is not an existing folder.")
    
dict_output_file_paths = utility.get_dict_with_output_file_paths_based_on_workingDir(workingDir)
args_file = dict_output_file_paths.get("args_file")
configuration_object_file = dict_output_file_paths.get("configuration_object_file")
nodes_file = dict_output_file_paths.get("nodes_file")
df_all_index_parameter_file = dict_output_file_paths.get("df_all_index_parameter_file")
df_all_simulations_file = dict_output_file_paths.get("df_all_simulations_file")
time_info_file = dict_output_file_paths.get("time_info_file")

dict_output_file_paths_qoi = utility.get_dict_with_qoi_name_specific_output_file_paths_based_on_workingDir(workingDir, qoi_string)
statistics_dictionary_file = dict_output_file_paths_qoi.get("statistics_dictionary_file")

dict_output_file_paths_qoi_time = utility.get_dict_with_qoi_name_timestamp_specific_output_file_paths_based_on_workingDir(workingDir, qoi_string, timestamp)
gpce_surrogate_file = dict_output_file_paths_qoi_time.get("gpce_surrogate_file")
gpce_coeffs_file = dict_output_file_paths_qoi_time.get("gpce_coeffs_file")

# Additional files being saved
sobol_m_error_file = workingDir / "sobol_m_error.npy"
sobol_m_qoi_file = workingDir / "sobol_m_qoi_file.npy"
sobol_t_error_file = workingDir / "sobol_t_error.npy"
sobol_t_qoi_file = workingDir / "sobol_t_qoi_file.npy"

In [ ]:
gpce_surrogate_file.is_file()

In [ ]:
dict_with_results_of_interest = defaultdict()

## Reading UQEF-Dynamic Output files

In [ ]:
with open(args_file, 'rb') as f:
    uqsim_args = pickle.load(f)
uqsim_args_dict = vars(uqsim_args)

with open(configuration_object_file, 'rb') as f:
    configurationObject = dill.load(f)
configurationObject

In [ ]:
uqsim_args_dict

In [ ]:
param_labeles = utility.get_list_of_uncertain_parameters_from_configuration_dict(
    configurationObject, raise_error=True, uq_method=uqsim_args_dict["uq_method"])
param_labeles

In [ ]:
variant = None
if uqsim_args_dict["uq_method"]== "mc":
    variant = "m1"
    dict_with_results_of_interest["mc_numevaluations"] = uqsim_args_dict["mc_numevaluations"]
    dict_with_results_of_interest["sampling_rule"] = uqsim_args_dict["sampling_rule"]
elif uqsim_args_dict["uq_method"]=="saltelli":
    variant = "m2"
    dict_with_results_of_interest["mc_numevaluations"] = uqsim_args_dict["mc_numevaluations"]
    dict_with_results_of_interest["sampling_rule"] = uqsim_args_dict["sampling_rule"]
elif uqsim_args_dict["uq_method"]=="sc" and uqsim_args_dict["regression"]:
    variant = "m3"
    dict_with_results_of_interest["q_order"] = uqsim_args_dict["sc_q_order"]
    dict_with_results_of_interest["q_order"] = uqsim_args_dict["sc_p_order"]
elif uqsim_args_dict["uq_method"]=="sc" and not uqsim_args_dict["regression"]:
    """
    [m4] gPCE+PSP with a full grid and polynomials of total-order
    [m5] gPCE+PSP with sparse grid and polynomials of total-order
    [m6] gPCE+PSP with a full grid and sparse polynomials (hyperbolic truncation)
    [m7] gPCE+PSP with sparse grid and sparse polynomials (hyperbolic truncation)
    """
    dict_with_results_of_interest["q_order"] = uqsim_args_dict["sc_q_order"]
    dict_with_results_of_interest["p_order"] = uqsim_args_dict["sc_p_order"]
    dict_with_results_of_interest["read_nodes_from_file"] = uqsim_args_dict["read_nodes_from_file"]
    dict_with_results_of_interest["sc_quadrature_rule"] = uqsim_args_dict["sc_quadrature_rule"]

    if (not uqsim_args_dict["sc_sparse_quadrature"] and not uqsim_args_dict["read_nodes_from_file"]) and uqsim_args_dict["cross_truncation"]==1.0:
        variant = "m4"
    elif (uqsim_args_dict["sc_sparse_quadrature"] or uqsim_args_dict["read_nodes_from_file"]) and uqsim_args_dict["cross_truncation"]==1.0:
        variant = "m5"
    elif (not uqsim_args_dict["sc_sparse_quadrature"] and not uqsim_args_dict["read_nodes_from_file"]) and uqsim_args_dict["cross_truncation"]<1.0:
        dict_with_results_of_interest["cross_truncation"] = uqsim_args_dict["cross_truncation"]
        variant = "m6"
    elif (uqsim_args_dict["sc_sparse_quadrature"] or uqsim_args_dict["read_nodes_from_file"]) and uqsim_args_dict["cross_truncation"]<1.0:
        dict_with_results_of_interest["cross_truncation"] = uqsim_args_dict["cross_truncation"]
        variant = "m7"
        
dict_with_results_of_interest["variant"] = variant
dict_with_results_of_interest["outputModelDir"] = workingDir

#     "error_model_linf":None, "error_model_l2":None, "error_mean":None, "error_var":None, 
#     "sobol_m_error":None, "sobol_t_error":None


In [ ]:
simulation_settings_dict = utility.read_simulation_settings_from_configuration_object(configurationObject)
qoi_column = simulation_settings_dict["qoi_column"]
qoi_column
list_qoi_column = simulation_settings_dict["list_qoi_column"]
list_qoi_column

In [ ]:
simulation_settings_dict

In [ ]:
with open(time_info_file) as f:
    lines = f.readlines()
    for line in lines:
        print(line)
        if line.startswith("time_model_simulations"):
         dict_with_results_of_interest["time_model_simulations"] = line.split(':')[1].strip()
        elif line.startswith("time_computing_statistics"):
            dict_with_results_of_interest["time_computing_statistics"] = line.split(':')[1].strip()


## Reading Simulation Nodes / Parameters

In [ ]:
with open(nodes_file, 'rb') as f:
#     simulationNodes = dill.load(f)
    simulationNodes = pickle.load(f)
# simulationNodes

print(simulationNodes.nodes.shape)
print(simulationNodes.parameters.shape)

dict_with_results_of_interest["number_full_model_evaluations"] = simulationNodes.nodes.shape[1]

In [ ]:
simulationNodes.joinedDists

In [ ]:
simulationNodes.joinedStandardDists

In [ ]:
if df_all_index_parameter_file.is_file():
    df_index_parameter = pd.read_pickle(df_all_index_parameter_file, compression="gzip")
else:
    df_index_parameter = None
df_index_parameter

In [ ]:
if df_index_parameter is not None:
    params_list = utility._get_parameter_columns_df_index_parameter_gof(
        df_index_parameter)
else:
    params_list = []
    for single_param in configurationObject["parameters"]:
        params_list.append(single_param["name"])
params_list

In [ ]:
fig = utility.plot_subplot_params_hist_from_df(df_index_parameter)
fig.update_layout(title="Prior Distribution of the Parameters",)
fig.show()

In [ ]:
df_nodes = utility.get_df_from_simulationNodes(simulationNodes, nodes_or_paramters="nodes", params_list=params_list)
df_nodes_params = utility.get_df_from_simulationNodes(simulationNodes, nodes_or_paramters="parameters",  params_list=params_list)


In [ ]:
df_nodes_params

In [ ]:
fig = utility.plot_3d_dynamic(df_nodes_params)
fig.show()

In [ ]:
utility.plot_2d_matrix_static(df_nodes, nodes_or_paramters="nodes")
# sns.set(style="ticks", color_codes=True)
# g = sns.pairplot(df_nodes, vars=list(df_nodes.columns), corner=True)
# plt.title(f"Plot: nodes", loc='left')
# plt.show()

In [ ]:
print(df_nodes_params.loc[728])
print(df_index_parameter.loc[728])

In [ ]:
if dict_with_results_of_interest["variant"] in ["m4", "m5", "m6", "m7"]:
    dict_with_results_of_interest["full_number_quadrature_points"] = \
    (dict_with_results_of_interest["q_order"] + 1) ** simulationNodes.nodes.shape[0]

## Reading all the simulations

In [ ]:
read_all_saved_simulations_file = True
if read_all_saved_simulations_file and df_all_simulations_file.is_file():
    # Reading Saved Simulations - Note: This migh be a huge file,
    # especially for MC/Saltelli kind of simulations
    df_simulation_result = pd.read_pickle(df_all_simulations_file, compression="gzip")
    dict_with_results_of_interest["number_full_model_evaluations"] = len(df_simulation_result)
else:
    df_simulation_result = None
df_simulation_result

## Read a Dictionary Containing Statistics Data

In [ ]:
statistics_dictionary = uqPostprocessing.read_all_saved_statistics_dict(\
    workingDir, list_qoi_column, uqsim_args_dict.get("instantly_save_results_for_each_time_step", False), throw_error=True)

statistics_dictionary  = statistics_dictionary['Value'][0.0]
statistics_dictionary

## Comparing E and Var

In [ ]:
numSamples = 10 ** 5  # numSamples_for_checking
rule = sampling_rule = "random"
compute_var = True
analytical_mean = 3.48227783540168 #None
analytical_var = 13.887058470972093 #None
if analytical_mean is None:
    expectedInterp, varianceInterp = utility.compute_mc_quantity(
        model=IshigamiModel.ishigami_func, jointDists=simulationNodes.joinedDists, numSamples=numSamples, rule=rule, 
        compute_mean=True, compute_var=compute_var)
    analytical_mean = expectedInterp
    analytical_var = varianceInterp

#dict_with_results_of_interest["analytical_mean"] = analytical_mean
dict_with_results_of_interest["error_mean"] = abs(analytical_mean - statistics_dictionary['E'])
print(f"analytical_mean = {analytical_mean} \n"
      f"approximated_mean = {statistics_dictionary['E']} \n"
f"Error in mean = {dict_with_results_of_interest['error_mean']} \n")

#dict_with_results_of_interest["analytical_var"] = analytical_var
dict_with_results_of_interest["error_var"] = abs(analytical_var - statistics_dictionary['Var'])
if analytical_var is not None:
    print(f"analytical_var = {analytical_var} \n"
         f"approximated_var = {statistics_dictionary['Var']} \n"
    f"Error in var = {dict_with_results_of_interest['error_var']} \n")


## Reading all SA related data, e.g., error

In [ ]:
Sobol_m_analytical = np.array([0.3138, 0.4424, 0.0], dtype=np.float64)
Sobol_m_analytical_2 = np.array([0.3139, 0.4424, 0.0000], dtype=np.float64)

Sobol_t_analytical = np.array([0.5574, 0.4424, 0.2436], dtype=np.float64)
Sobol_t_analytical_2 = np.array([0.5576, 0.4424, 0.2437], dtype=np.float64)

In [ ]:
if "Sobol_m" in statistics_dictionary and sobol_m_qoi_file.is_file():
    Sobol_m = statistics_dictionary['Sobol_m'] 
    print(f"Sobol_m - {Sobol_m}")
    Sobol_m = np.load(sobol_m_qoi_file)
    print(f"Sobol_m - {Sobol_m}")
    Sobol_m_error = abs(Sobol_m - Sobol_m_analytical)
    Sobol_m_error_2 = abs(np.load(sobol_m_error_file))
    print(f"Sobol_m_error - {Sobol_m_error}")
    print(f"Sobol_m_error_2 - {Sobol_m_error_2}")
    dict_with_results_of_interest["sobol_m"] = Sobol_m
    dict_with_results_of_interest["sobol_m_error"] = Sobol_m_error_2
if "Sobol_t" in statistics_dictionary and sobol_t_qoi_file.is_file():
    Sobol_t = statistics_dictionary['Sobol_m'] 
    print(f"Sobol_t - {Sobol_t}")
    Sobol_t = np.load(sobol_t_qoi_file)
    print(f"Sobol_t - {Sobol_t}")
    Sobol_t_error = abs(Sobol_t - Sobol_t_analytical)
    Sobol_t_error_2 = abs(np.load(sobol_t_error_file))
    print(f"Sobol_t_error - {Sobol_t_error}")
    print(f"Sobol_t_error_2 - {Sobol_t_error_2}")
    dict_with_results_of_interest["sobol_t"] = Sobol_t
    dict_with_results_of_interest["sobol_t_error"] = Sobol_t_error_2

## GPCE Surrogate (re-evaluating it...)

In [ ]:
print(gpce_surrogate_file.is_file())
if gpce_surrogate_file.is_file():
    with open(gpce_surrogate_file, 'rb') as f:
    #     simulationNodes = dill.load(f)
        gpce_surrogate = pickle.load(f)
elif 'gPCE' in statistics_dictionary:
    gpce_surrogate = statistics_dictionary['gPCE']
else:
    raise Exception(f"Sorry you did not save gPCE surrogate")

dict_with_results_of_interest["max_p_order"] = max(np.linalg.norm(vector, ord=1) for vector in gpce_surrogate.exponents)
dict_with_results_of_interest["gPCE_num_coeffs"] = gpce_surrogate.exponents.shape[0]

In [ ]:
gpce_coeffs = np.load(gpce_coeffs_file, allow_pickle=True)
print(gpce_coeffs)

In [ ]:
gpce_surrogate

In [ ]:
gpce_surrogate.coefficients

In [ ]:
gpce_surrogate.todict()

In [ ]:
gpce_surrogate.exponents

In [ ]:
# visualize
indices = gpce_surrogate.exponents
dimensionality = indices.shape[1]
number_of_terms = indices.shape[0]
dict_for_plotting = {f"q_{i+1}":indices[:, i] for i in range(dimensionality)}
df_nodes_weights = pd.DataFrame(dict_for_plotting)
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(df_nodes_weights, vars = list(dict_for_plotting.keys()), corner=True)
# plt.title(title, loc='left')
plt.show()

In [ ]:
# Generating new samples
numSamples = 1000 #5**dim #10**dim  # Note: Big Memory problem when more than 10**4 points?
rule = 'r'

new_set_of_parameters = simulationNodes.joinedDists.sample(size=numSamples, rule=rule)
new_set_of_nodes_transformed = utility.transformation_of_parameters(
    new_set_of_parameters, simulationNodes.joinedDists, simulationNodes.joinedStandardDists)

# some check-up
new_set_of_nodes = simulationNodes.joinedStandardDists.sample(size=numSamples, rule=rule)
new_set_of_parameters_transformed = utility.transformation_of_parameters(
    new_set_of_nodes, simulationNodes.joinedStandardDists, simulationNodes.joinedDists)

new_set_of_parameters_II = utility.transformation_of_parameters(
    new_set_of_nodes_transformed, simulationNodes.joinedStandardDists, simulationNodes.joinedDists)
np.allclose(new_set_of_parameters, new_set_of_parameters_II, atol=0.01)

In [ ]:
# Generating new samples
numSamples = 1000 #5**dim #10**dim  # Note: Big Memory problem when more than 10**4 points?
rule = 'r'

new_set_of_parameters = simulationNodes.joinedDists.sample(size=numSamples, rule=rule)
new_set_of_nodes_transformed = utility.transformation_of_parameters(
    new_set_of_parameters, simulationNodes.joinedDists, simulationNodes.joinedStandardDists)


start = time.time()
# gPCE_model_evaluated = gPCE_model(*new_set_of_nodes_transformed)
gPCE_model_evaluated = gpce_surrogate(*new_set_of_nodes_transformed)
end = time.time()
reevaluation_surrogate_model_time = end - start
print(f"Time needed for evaluating {new_set_of_nodes_transformed.shape[1]} \
gPCE model is: {reevaluation_surrogate_model_time}")


start = time.time()
original_model_evaluated = np.empty([new_set_of_parameters.shape[1],])
i=0
for single_sample in new_set_of_parameters.T:
    original_model_evaluated[i] = IshigamiModel.ishigami_func(
        single_sample, a_model_param=configurationObject['other_model_parameters']['a'], b_model_param=configurationObject['other_model_parameters']['b'])
    i+=1
end = time.time()
reevaluation_model_time = end - start
print(f"Time needed for evaluating {new_set_of_nodes_transformed.shape[1]} \
of the original model is: {reevaluation_model_time}")

error_linf = np.max(np.abs(original_model_evaluated - gPCE_model_evaluated))
# error_linf_np = np.linalg.norm(original_model_evaluated - gPCE_model_evaluated, ord=np.inf)

error_l2 = np.sqrt(np.sum((original_model_evaluated - gPCE_model_evaluated)**2))
# error_l2_np = np.linalg.norm(original_model_evaluated - gPCE_model_evaluated, ord=2)
error_l2_scaled = np.sqrt(np.sum((original_model_evaluated - gPCE_model_evaluated)**2)) / math.sqrt(numSamples)

print(f"Linf Error = {error_linf};")
print(f"L2 Error = {error_l2}; L2 Error scaled = {error_l2_scaled}")

dict_with_results_of_interest["comparison_surrogate_vs_model_numSamples"] = numSamples
dict_with_results_of_interest["comparison_surrogate_vs_model_mc_rule"] = rule
dict_with_results_of_interest["reevaluation_surrogate_model_duration"] = reevaluation_surrogate_model_time
dict_with_results_of_interest["reevaluation_model_duration"] = reevaluation_model_time
dict_with_results_of_interest["error_model_linf"] = error_linf
dict_with_results_of_interest["error_model_l2"] = error_l2
dict_with_results_of_interest["error_model_l2_scaled"] = error_l2_scaled

In [ ]:
print(gPCE_model_evaluated.shape)
print(original_model_evaluated.shape)

In [ ]:
gPCE_model_evaluated

In [ ]:
original_model_evaluated

In [ ]:
error_array = gPCE_model_evaluated - original_model_evaluated
error_array

In [ ]:
# jet another option is to re-evaluate the model in the original set of nodes
# reading gPCE model

# df_nodes_params; df_nodes
# df_index_parameter

f = np.empty([len(df_index_parameter[["x1","x2","x3",]]),])
gpce_surrogate_eval = np.empty([len(df_index_parameter[["x1","x2","x3",]]),])

for index, row in df_index_parameter[["x1","x2","x3",]].iterrows():
    single_sample = list(row.values)
    single_sample_standard = utility.transformation_of_parameters(
        single_sample, simulationNodes.joinedDists, simulationNodes.joinedStandardDists)
    
    f[index] = IshigamiModel.ishigami_func(
        single_sample, a_model_param=configurationObject['other_model_parameters']['a'], b_model_param=configurationObject['other_model_parameters']['b'])
    gpce_surrogate_eval[index] = gpce_surrogate(*single_sample_standard)  # gpce_surrogate

df_index_parameter['f'] = f
df_index_parameter['gPCE'] = gpce_surrogate_eval
df_index_parameter['error'] = gpce_surrogate_eval - f
df_index_parameter

error_linf = np.max(np.abs(df_index_parameter['f'] - df_index_parameter['gPCE']))
error_linf_np = np.linalg.norm(df_index_parameter['f'] - df_index_parameter['gPCE'], ord=np.inf)

error_l2 = np.sqrt(np.sum((df_index_parameter['f'] - df_index_parameter['gPCE'])**2))
error_l2_np = np.linalg.norm(df_index_parameter['f'] - df_index_parameter['gPCE'], ord=2)
error_l2_scaled = np.sqrt(np.sum((df_index_parameter['f'] - df_index_parameter['gPCE'])**2)) / math.sqrt(len(df_index_parameter))

print(f"Linf Error = {error_linf}; Linf Error np = {error_linf_np} \n")
print(f"L2 Error = {error_l2}; L2 Error np = {error_l2_np}; L2 Error scaled = {error_l2_scaled}")

## Printing the final dict_with_results_of_interest

In [ ]:
# an example of pce output
dict_with_results_of_interest

# Examples of reading some saved Runs/Files and printing the dictionary with all the relevant information
## Relevant for producing convergence graphs...

In [ ]:
# TODO - change these paths/variables accordingly
scratch_dir = pathlib.Path("/work/ga45met/ishigami_runs/simulations_sep_2024")
workingDir = scratch_dir / "sc_full_p7_q14"
qoi_string="Value"
timestamp=0.0

In [ ]:
a = 7
b = 0.1
ishigamiModelObject = IshigamiModel.IshigamiModel(configurationObject=None, a=a, b=b)

In [ ]:
dict_with_results_of_interest = uqPostprocessing.read_all_saved_uqef_dynamic_results_and_produce_dict_of_interest_single_qoi_single_timestamp(
    workingDir=workingDir, 
    timestamp=timestamp, qoi_column_name=qoi_string,
    plotting=False, model=ishigamiModelObject,
    analytical_E=3.48227783540168,
    analytical_Var=13.887058470972093,
    analytical_Sobol_t=np.array([0.5574, 0.4424, 0.2436], dtype=np.float64),
    analytical_Sobol_m=np.array([0.3138, 0.4424, 0.0], dtype=np.float64),
    compare_surrogate_and_original_model=True
)
dict_with_results_of_interest

In [ ]:
scratch_dir = pathlib.Path("/work/ga45met/ishigami_runs/simulations_sep_2024")
workingDir = scratch_dir / "mc_100000_random"
qoi_string="Value"
timestamp=0.0
dict_with_results_of_interest = uqPostprocessing.read_all_saved_uqef_dynamic_results_and_produce_dict_of_interest_single_qoi_single_timestamp(
    workingDir=workingDir, 
    timestamp=timestamp, qoi_column_name=qoi_string,
    plotting=False, model=ishigamiModelObject,
    analytical_E=3.48227783540168,
    analytical_Var=13.887058470972093,
    analytical_Sobol_t=np.array([0.5574, 0.4424, 0.2436], dtype=np.float64),
    analytical_Sobol_m=np.array([0.3138, 0.4424, 0.0], dtype=np.float64),
    compare_surrogate_and_original_model=True
)
dict_with_results_of_interest

In [ ]:
scratch_dir = pathlib.Path("/work/ga45met/ishigami_runs/simulations_sep_2024")
workingDir = scratch_dir / "saltelli_1000_random"
qoi_string="Value"
timestamp=0.0
dict_with_results_of_interest = uqPostprocessing.read_all_saved_uqef_dynamic_results_and_produce_dict_of_interest_single_qoi_single_timestamp(
    workingDir=workingDir, 
    timestamp=timestamp, qoi_column_name=qoi_string,
    plotting=False, model=ishigamiModelObject,
    analytical_E=3.48227783540168,
    analytical_Var=13.887058470972093,
    analytical_Sobol_t=np.array([0.5574, 0.4424, 0.2436], dtype=np.float64),
    analytical_Sobol_m=np.array([0.3138, 0.4424, 0.0], dtype=np.float64),
    compare_surrogate_and_original_model=True
)
dict_with_results_of_interest

# PCE Analysis of Ishigami function using utility functions from UQEF-Dynamic
This is something extra; I wanted to double check a couple of different approaches for builidng gPCE surrogate model

In [ ]:
a = 7
b = 0.1
ishigamiModelObject = IshigamiModel.IshigamiModel(configurationObject=None, a=a, b=b)

x1 = cp.Uniform(-math.pi, math.pi)
x2 = cp.Uniform(-math.pi, math.pi)
x3 = cp.Uniform(-math.pi, math.pi)

joint_isghigami = cp.J(x1, x2, x3)
joint_isghigami_standard = cp.J(cp.Uniform(-1,1), cp.Uniform(-1,1), cp.Uniform(-1,1))

joint_dist_standard = joint_isghigami_standard
joint_dist = joint_isghigami

# this is 'my' standard approach, where in the background we always sample from and build surrogate models 
# that fit the standard distribution, and then do parameter transformation to generate parameter values to stimulate the model
sampleFromStandardDist = True
gPCE_over_time_my_approach, polynomial_expansion_my_approach, \
norms_my_approach, coeff_my_approach = uqPostprocessing.compute_PSP_for_uqef_dynamic_model(
    ishigamiModelObject, joint_dist, \
    quadrature_order=14, expansion_order=7, 
    sampleFromStandardDist=sampleFromStandardDist,
    joint_dist_standard=joint_dist_standard,
    rule_quadrature='g', \
    poly_rule='three_terms_recurrence', poly_normed=True, \
    qoi_column_name=ishigamiModelObject.qoi_column, 
    time_column_name=ishigamiModelObject.time_column_name, 
    index_column_name=ishigamiModelObject.index_column_name
    )

# This is an approach without sampling from a 'standard' site. and transformation step, instead everything is
# performed directly in the original stochastic space/range and with original distribution.
sampleFromStandardDist = False
gPCE_over_time, polynomial_expansion, norms, coeff = uqPostprocessing.compute_PSP_for_uqef_dynamic_model(
    ishigamiModelObject, joint_dist, \
    quadrature_order=14, expansion_order=7, 
    sampleFromStandardDist=sampleFromStandardDist,
    joint_dist_standard=joint_dist_standard,
    rule_quadrature='g', \
    poly_rule='three_terms_recurrence', poly_normed=True, \
    qoi_column_name=ishigamiModelObject.qoi_column, 
    time_column_name=ishigamiModelObject.time_column_name, 
    index_column_name=ishigamiModelObject.index_column_name
    )

In [ ]:
joint_dist_standard = joint_isghigami_standard
joint_dist = joint_isghigami

nodes_quad, weights_quad = utility.generate_quadrature_nodes_and_weights(joint_dist_standard, quadrature_order=7)
parameters_quad = utility.generate_parameters_from_nodes(nodes_quad, joint_dist_standard, joint_dist)

parameters_quad_orig, weights_quad_orig = utility.generate_quadrature_nodes_and_weights(joint_dist, quadrature_order=7)

np.allclose(parameters_quad, parameters_quad_orig, atol=0.01)


In [ ]:
# Comparing these two approaches

# Generating new samples
numSamples = 1000 #5**dim #10**dim  # Note: Big Memory problem when more than 10**4 points?
rule = 'r'

new_set_of_parameters = joint_dist.sample(size=numSamples, rule=rule)
new_set_of_nodes_transformed = utility.transformation_of_parameters(
    new_set_of_parameters, joint_dist, joint_dist_standard)

# new_set_of_nodes = joint_dist_standard.sample(size=numSamples, rule=rule)
# new_set_of_parameters_transformed = utility.transformation_of_parameters(
#     new_set_of_nodes, joint_dist_standard, joint_dist)

gPCE_model_evaluated_original = gPCE_over_time[0](*new_set_of_parameters)
gPCE_model_evaluated_my_approach = gPCE_over_time_my_approach[0](*new_set_of_nodes_transformed)
# gPCE_model_evaluated_ionut = gPCE_over_time_ionut[0](*new_set_of_parameters)

original_model_evaluated = np.empty([new_set_of_parameters.shape[1],])
i=0
for single_sample in new_set_of_parameters.T:
    original_model_evaluated[i] = IshigamiModel.ishigami_func(
        single_sample, a_model_param=a, b_model_param=b)
    i+=1

np.allclose(gPCE_model_evaluated_original, gPCE_model_evaluated_my_approach, atol=0.01)

In [ ]:
error_linf = np.max(np.abs(original_model_evaluated - gPCE_model_evaluated_original))
error_l2_scaled = np.sqrt(np.sum((original_model_evaluated - gPCE_model_evaluated_original)**2)) / math.sqrt(numSamples)
print(f"gPCE_model_evaluated_original - Linf Error = {error_linf};")
print(f"gPCE_model_evaluated_original - L2 Error scaled = {error_l2_scaled}")

## Checking if MC computations work...

In [ ]:
a = 7
b = 0.1
ishigamiModelObject = IshigamiModel.IshigamiModel(configurationObject=None, a=a, b=b)

x1 = cp.Uniform(-math.pi, math.pi)
x2 = cp.Uniform(-math.pi, math.pi)
x3 = cp.Uniform(-math.pi, math.pi)

joint_isghigami = cp.J(x1, x2, x3)
joint_isghigami_standard = cp.J(cp.Uniform(-1,1), cp.Uniform(-1,1), cp.Uniform(-1,1))

joint_dist_standard = joint_isghigami_standard
joint_dist = joint_isghigami
read_nodes_from_file = False
sampleFromStandardDist = True

E_over_time, Var_over_time, StdDev_over_time, \
Skew_over_time, Kurt_over_time, P10_over_time, P90_over_time, sobol_m_over_time = \
uqPostprocessing.run_uq_mc_sim_and_compute_mc_stat_for_uqef_dynamic_model(
    model=ishigamiModelObject,
    jointDists=joint_dist, jointStandard=joint_dist_standard, numSamples=1000, rule="R",
    sampleFromStandardDist=sampleFromStandardDist,
    read_nodes_from_file=False, 
    rounding=False, round_dec=4,
    qoi_column_name=ishigamiModelObject.qoi_column, 
    time_column_name=ishigamiModelObject.time_column_name, 
    index_column_name=ishigamiModelObject.index_column_name,
    return_dict_over_timestamps=False,
    compute_mean=True, compute_var=True, compute_std=True,
    compute_skew=True,
    compute_kurt=True,
    compute_p10=True,
    compute_p90=True,
    compute_Sobol_m=True,
)

In [ ]:
E_over_time

In [ ]:
sobol_m_over_time